In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
! pip install transformers

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np

In [ ]:
def get_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

In [ ]:
model = AutoModel.from_pretrained('BM-K/KoSimCSE-bert-multitask')  
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-bert-multitask')  

In [ ]:
data = pd.read_csv('./Books.csv', encoding = 'utf-8')

In [ ]:
overview = '' #책소개를 넣어주세요.

In [ ]:
def recommend(overview, top = 10) :

    score = []

    for i in range(len(data)) :
        tmp_sentence = [overview, data['overview'][i]]
        inputs = tokenizer(tmp_sentence, padding = True, truncation = True, return_tensors = 'pt')
        embeddings, _ = model(**inputs, return_dict = False)
        score = get_score(embeddings[0][0], embeddings[1][0])[0][0].item()
        score.append(score)

    score = sorted(score, reverse = True)
    score = [(data['title'][i], score) for i, score in enumerate(score[0:top])]

    return score